In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/BIO_tagged_dataset_Modified2.csv')

In [ ]:
data

,Unnamed: 0,Unnamed: 0.1,text,keywords,BIO_tag
0,0,0,several inorganic flocculating agents includin...,"['Al2(SO4)3,', 'analyzes the main factors', 'C...",O B I I O B B O O B I I B O O O B I I O O O O ...
1,1,1,video oculography is one of eye movement measu...,"['accurately estimate the pupil center', 'Cann...",B I O O O B I I O O O O B O O B I I I I O B I ...
2,2,2,the first principles calculations are performe...,"['Al', 'alloy', 'Brillouin-zone sampling', 'Ca...",O B I I O O O O B I I I I O O O B I O O O O B ...
3,3,3,microhardness can be related to other macrosco...,"['compression testing', 'derived a direct prop...",B O O O O O O O O O O O O O O O O O O B I O B ...
4,4,4,we consider finite time future or big rip sing...,['back reaction of conformal quantum fields ne...,O O B I O O O O O O O O O O B I I O O O O O O ...
...,...,...,...,...,...
488,488,488,with development of performance based design s...,"['aspects of fire risk analysis', 'average fir...",O O O B I I O O O O O O O O O O O O O O O O B ...
489,489,489,elrs are particularly attractive for the synth...,"['aggregation', 'block copolymers', 'cytotoxic...",B O O O O O O O B I O O O O B O O O O O O O O ...
490,490,490,this paper proposes sentence stress feedback s...,"['CALL system', 'predict and detect the senten...",O O O B I I I O O O O O O O O O O O O O O B I ...
491,491,491,arrays of tfts and circuits were fabricated on...,"['aluminium gate electrodes', 'ambient air', '...",O O B O B O O O O O O B I B O O O O B O O O O ...


In [ ]:
len(data)

493

In [ ]:

data.iloc[[0]]['text']

0    several inorganic flocculating agents includin...
Name: text, dtype: object

In [ ]:
import nltk
from nltk import word_tokenize
import string
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import numpy as np
import math  

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
textRank_keywords=[]

In [ ]:
def buildGraph(vocabulary , processed_text , lemmatized_text , stopwords_plus):
  vocab_len = len(vocabulary)

  weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

  score = np.zeros((vocab_len),dtype=np.float32)
  window_size = 3
  covered_coocurrences = []

  for i in range(0,vocab_len):
      score[i]=1
      for j in range(0,vocab_len):
          if j==i:
              weighted_edge[i][j]=0
          else:
              for window_start in range(0,(len(processed_text)-window_size)):
                
                  window_end = window_start+window_size
                
                  window = processed_text[window_start:window_end]
                
                  if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                      index_of_i = window_start + window.index(vocabulary[i])
                      index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                      if [index_of_i,index_of_j] not in covered_coocurrences:
                          weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                          covered_coocurrences.append([index_of_i,index_of_j])
  inout = np.zeros((vocab_len),dtype=np.float32)

  for i in range(0,vocab_len):
      for j in range(0,vocab_len):
          inout[i]+=weighted_edge[i][j]
  MAX_ITERATIONS = 50
  d=0.85
  threshold = 0.0001 #convergence threshold

  for iter in range(0,MAX_ITERATIONS):
      prev_score = np.copy(score)
    
      for i in range(0,vocab_len):
        
          summation = 0
          for j in range(0,vocab_len):
              if weighted_edge[i][j] != 0:
                  summation += (weighted_edge[i][j]/inout[j])*score[j]
                
          score[i] = (1-d) + d*(summation)
    
      if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
          # print("Converging at iteration "+str(iter)+"....")
          break
  phrases = []

  phrase = " "
  for word in lemmatized_text:
    
      if word in stopwords_plus:
          if phrase!= " ":
              phrases.append(str(phrase).strip().split())
          phrase = " "
      elif word not in stopwords_plus:
         phrase+=str(word)
         phrase+=" "

  # print("Partitioned Phrases (Candidate Keyphrases): \n")
  # print(phrases)
  unique_phrases = []

  for phrase in phrases:
      if phrase not in unique_phrases:
          unique_phrases.append(phrase)

  # print("Unique Phrases (Candidate Keyphrases): \n")
  # print(unique_phrases)
  for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            #if len(phrase)>1 then the current phrase is multi-worded.
            #if the word in vocabulary is present in unique_phrases as a single-word-phrase
            # and at the same time present as a word within a multi-worded phrase,
            # then I will remove the single-word-phrase from the list.
            unique_phrases.remove([word])
            
  # print("Thinned Unique Phrases (Candidate Keyphrases): \n")
  # print(unique_phrases) 
  phrase_scores = []
  keywords = []
  for phrase in unique_phrases:
      phrase_score=0
      keyword = ''
      for word in phrase:
          keyword += str(word)
          keyword += " "
          phrase_score+=score[vocabulary.index(word)]
      phrase_scores.append(phrase_score)
      keywords.append(keyword.strip())
  print(keywords)
  temp=str(keywords)
  textRank_keywords.append(temp)
  

In [ ]:
def TextRank(Text):
  def clean(text):
    text = text.lower()
    printable = set(string.printable)
    text = filter(lambda x: x in printable, text)
    text = "".join(list(text))
    return text

  Cleaned_text = clean(Text)
  # print(Cleaned_text)
  text = word_tokenize(Cleaned_text)

  # print ("Tokenized Text: \n")
  # print (text)
  POS_tag = nltk.pos_tag(text)

  # print ("Tokenized Text with POS tags: \n")
  # print (POS_tag)

  wordnet_lemmatizer = WordNetLemmatizer()

  adjective_tags = ['JJ','JJR','JJS']

  lemmatized_text = []

  for word in POS_tag:
      if word[1] in adjective_tags:
          lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
      else:
          lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
        
  # print ("Text tokens after lemmatization of adjectives and nouns: \n")
  # print (lemmatized_text)
  POS_tag = nltk.pos_tag(lemmatized_text)

  # print ("Lemmatized text with POS tags: \n")
  # print (POS_tag)
  stopwords = []

  wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 

  for word in POS_tag:
      if word[1] not in wanted_POS:
          stopwords.append(word[0])

  punctuations = list(str(string.punctuation))

  stopwords = stopwords + punctuations
  stopword_file = open("/content/drive/MyDrive/long_stopwords.txt", "r")
  #Source = https://www.ranks.nl/stopwords

  lots_of_stopwords = []

  for line in stopword_file.readlines():
      lots_of_stopwords.append(str(line.strip()))

  stopwords_plus = []
  stopwords_plus = stopwords + lots_of_stopwords
  stopwords_plus = set(stopwords_plus)
  processed_text = []
  for word in lemmatized_text:
      if word not in stopwords_plus:
          processed_text.append(word)
  # print (processed_text)
  vocabulary = list(set(processed_text))
  # print (vocabulary)
  buildGraph(vocabulary , processed_text, lemmatized_text , stopwords_plus)

  

In [ ]:
text='Several inorganic flocculating agents, including FeSO4, Al2(SO4)3, FeCl3 and an organic coagulant aid PAM, were used to treat the wastewater from domestic anima and poultry breeding in this paper. The ideal operating conditions were attained by single factor experiment and orthogonal design experiment. And the ideal operating conditions are follows: the dose of FeSO4 and PAM is 135.2mg/L and 0.384mg/L respectively when keeping the pH 10; and the corresponding removal rate is 55% and 60% for COD and turbidity. Based on the experimental results, this paper analyzes the main factors that affect wastewater flocculation treatment.'

In [ ]:
for i in range(len(data)):
  my_str =str(data.iloc[i]['text'])

  TextRank(my_str)
  

['inorganic flocculating agent including feso4 fecl3', 'organic coagulant aid pam', 'wastewater', 'domestic anima', 'poultry breeding', 'paper', 'ideal operating condition', 'single factor experiment', 'orthogonal design experiment', 'dose', 'keeping', 'ph', 'removal rate', 'cod', 'turbidity', 'experimental result', 'main factor']
['video oculography', 'eye movement measurement method key problem', 'center', 'location method', 'morphology', 'canny algorithm', 'wifi', 'vog system', 'late work', 'healthy volunteer', 'tracking test', 'location method experimental result', 'eye movement', 'anticipated result']
['principle calculation', 'cambridge serial total energy package', 'plane', 'pseudopotential dft', 'exchange correlation functional', 'generalized gradient approximation', 'electron ion interaction', 'vanderbilt type ultrasoft pseudopotentials', 'basis set', 'cutoff', 'brillouin zone sampling wa', 'monkhorst pack scheme', 'point spacing', 'reciprocal space', 'test', 'computational pa

In [ ]:
listtemp=['apple' , 'pie' ,'jam']
my=str(listtemp)
print(my)

['apple', 'pie', 'jam']


In [ ]:
print(textRank_keywords)
print(len(textRank_keywords))
print(len(data))

["['inorganic flocculating agent including feso4 fecl3', 'organic coagulant aid pam', 'wastewater', 'domestic anima', 'poultry breeding', 'paper', 'ideal operating condition', 'single factor experiment', 'orthogonal design experiment', 'dose', 'keeping', 'ph', 'removal rate', 'cod', 'turbidity', 'experimental result', 'main factor']", "['video oculography', 'eye movement measurement method key problem', 'center', 'location method', 'morphology', 'canny algorithm', 'wifi', 'vog system', 'late work', 'healthy volunteer', 'tracking test', 'location method experimental result', 'eye movement', 'anticipated result']", "['principle calculation', 'cambridge serial total energy package', 'plane', 'pseudopotential dft', 'exchange correlation functional', 'generalized gradient approximation', 'electron ion interaction', 'vanderbilt type ultrasoft pseudopotentials', 'basis set', 'cutoff', 'brillouin zone sampling wa', 'monkhorst pack scheme', 'point spacing', 'reciprocal space', 'test', 'computat

In [ ]:
data['TR_Keyword']=textRank_keywords

In [ ]:
print(data.iloc[0]['TR_Keyword'])
print(data.iloc[0])

['inorganic flocculating agent including feso4 fecl3', 'organic coagulant aid pam', 'wastewater', 'domestic anima', 'poultry breeding', 'paper', 'ideal operating condition', 'single factor experiment', 'orthogonal design experiment', 'dose', 'keeping', 'ph', 'removal rate', 'cod', 'turbidity', 'experimental result', 'main factor']
Unnamed: 0                                                      0
Unnamed: 0.1                                                    0
text            several inorganic flocculating agents includin...
keywords        ['Al2(SO4)3,', 'analyzes the main factors', 'C...
BIO_tag         O B I I O B B O O B I I B O O O B I I O O O O ...
TR_Keyword      ['inorganic flocculating agent including feso4...
Name: 0, dtype: object


In [ ]:
from google.colab import files
data.to_csv("TextRank_Keywords.csv" , index=False)
files.download("TextRank_Keywords.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Third Method

In [ ]:
def top_keywords_sentences(text,stopwords=None, spacy_nlp=None, skip_ner=True, phrase_limit=15, sent_word_limit=150):
    #Parse incoming text
    parse=parse_doc(text2json(text))
    parse_list=[json.loads(pretty_print(i._asdict())) for i in parse]

    #Create and rank graph for keywords
    graph, ranks = text_rank(parse_list)
    norm_rank=normalize_key_phrases(parse_list, ranks, stopwords=stopwords, spacy_nlp=spacy_nlp, skip_ner=skip_ner)
    norm_rank_list=[json.loads(pretty_print(rl._asdict())) for rl in norm_rank ]
    phrases = ", ".join(set([p for p in limit_keyphrases(norm_rank_list, phrase_limit=phrase_limit)]))

    # return a matrix like result for the top keywords
    kernel = rank_kernel(norm_rank_list)

    # Rank the sentences
    top_sent=top_sentences(kernel, parse_list)
    top_sent_list=[json.loads(pretty_print(s._asdict())) for s in top_sent ]
    sent_iter = sorted(limit_sentences(top_sent_list, word_limit=sent_word_limit), key=lambda x: x[1])

    # Return ranked sentences
    s=[]
    for sent_text, idx in sent_iter:
        s.append(make_sentence(sent_text))

    graf_text = " ".join(s)

    return(graf_text,phrases)


In [ ]:
pip install pytextrank

     |████████████████████████████████| 1.1 MB 6.1 MB/s 
     |████████████████████████████████| 46 kB 4.4 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 6.0 MB 35.7 MB/s 
     |████████████████████████████████| 11.2 MB 38.6 MB/s 
     |████████████████████████████████| 895 kB 45.5 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 181 kB 57.6 MB/s 
     |████████████████████████████████| 10.1 MB 49.6 MB/s 
     |████████████████████████████████| 628 kB 51.0 MB/s 
     |████████████████████████████████| 451 kB 41.9 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing inst

In [ ]:
import xang_pytextrank as pyt
import time

text="Compatibility of systems of linear constraints \
over the set of natural numbers.\
Criteria of compatibility of a system of linear\
Diophantine equations,\
strict inequations, and nonstrict inequations are considered.\
Upper bounds for components of a minimal set of solutions and \
algorithms of construction of minimal generating sets of solutions\
for all types of systems are given. These criteria and the \
corresponding algorithms for constructing a minimal\
supporting set of solutions can be used in solving all \
the considered types systems and systems of mixed types."

start=time.time()
phrase,word=pyt.top_keywords_sentences(text,phrase_limit=15)
print('Keywords:',word)
print('Time Taken: ',time.time()-start)

ModuleNotFoundError: ignored